In [29]:
using HDF5;                     # for .hd5 file manipulation
using DifferentialEquations;    # Provides a variety of differential solvers
using LinearAlgebra: Diagonal;  # Efficient Diagonal matrixes
using Statistics;               # Let's get this fucking bread 
using DataFrames;               # For succinct data manipulation
using CSV;                      # For writing the data to prevent constant re-running
using ForwardDiff;              # For profiling the gradients 
using Gadfly;                   #! For testing

In [30]:
"""
Takes time series data and calculates the average of each year.
"""
function bin(time_series::Vector, solution_vector::Vector)::Vector
    local binned_solution = Vector(undef, 0);   # Setting a vector to hold the bins 
    local whole_times = @.floor(time_series);   # Creating a vector of discrete time.
    for whole_time in unique(whole_times)       # Looping over the unique elements discrete times 
        local indexes = findall(whole_times .== whole_time);        # Getting the indexes
        push!(binned_solution, mean(solution_vector[indexes]));   # Appending to binned_solution
    end
    return binned_solution
end

bin

In [31]:
"""
Reads the flux (amounts), production (projection) and reserviour contents from 
a .hd5 file with file_name. It returns the transfer operator and production 
projection 
"""
function read_hd5(file_name::String)::Tuple{Matrix{Float64}, Vector{Float64}}
    local hd5 = h5open(file_name);                      # Opening the HDF5 file
    local F = hd5["fluxes"][1:end, 1:end];              # Retrieving the flux matrix 
    local P = hd5["production coefficients"][1:end];    # Retrieving the projection of the production 
    local N = hd5["reservoir content"][1:end, 1:end];   # The C14 reserviour contents 
    close(hd5);                                         # Closing the file 

    local λ = Diagonal([log(2) / 5730 for i in 1:11]);          # Constructing the decay matrix
    F = transpose(F) ./ N;                                      # The proportion flux
    local TO = transpose(F) - Diagonal(vec(sum(F, dims=2))) - λ;# Construncting the transfer operator
    return TO, P                                           
end

read_hd5

In [32]:
"""
Passed a solver function runs the solver and returns the speed and binned data
"""
function run_solver(solver, ∇::Function, U0::Vector, p::Vector)::Vector
    local problem = ODEProblem(∇, U0, (760.0, 790.0), p);   # Creating the ODEProblem instance
    local solution = solve(problem, reltol = 1e-6, solver); # Solving the ODE  
    local time = Array(solution.t);                         # Storing the time sampling 
    solution = Array(solution)[2, 1:end];   # Storing the solution for troposphere 
    solution = bin(time, solution);         # Getting the annual means
    return solution;                        # Binning the results into years 
end

run_solver

In [33]:
"""
Takes a list of solvers as an input and runs a multithreaded comparison that 
stores the time information and the binned output of the ODE solver.
"""
function profile_solvers(solvers::Vector, ∇::Function, u0::Vector{Float64},
        p::Vector)::DataFrame
    local C14 = Matrix{Float64}(undef, length(solvers), 31);    # Creating the storage Matrix 
    local t_mean = Vector{Float64}(undef, length(solvers));     # For the mean of the times
    local t_var = Vector{Float64}(undef, length(solvers));      # For the time varience 
    local results = DataFrame(solver = @.string(solvers));      # DataFrame of summary Statistics

    for (index, solver) in enumerate(solvers)           # Looping over the solvers 
        local time_sample = Vector{Float64}(undef, 10); # A vector to hold the different run times of each trial 
        for i in 1:10
            local timer = time();                       # Starting a timer
            solution = run_solver(solver(), ∇, u0, p);  # Running the solver
            time_sample[i] = time() - timer;            # ending the timer 

            if i == 10                          # Storing final run
                C14[index, 1:end] = solution;   # filling C14
            end
        end
        t_mean[index] = mean(time_sample[2:end]);   # Storing run time ignoring compile run.
        t_var[index] = var(time_sample[2:end]);     # Storing time error
    end 

    C14 = (C14' .- median(C14, dims=1)')';   # Calculating deviations from median 
    C14 = @.abs(C14);   # Absolute value 
    results.accuracy = vec(mean(C14, dims=2));      # Calculating the mean of the deviation from the median 
    results.accuracy_var = vec(var(C14, dims=2));   # Calculating the RMSE error << is better 
    results.time_mean = t_mean;                     # Storing the mean run time
    results.time_var = t_var;                       # Storing the varience of the time sample 
    return results
end


profile_solvers

In [58]:
"""
Calculates the gradient using a χ² loss function. 
"""
function select_gradient_solver(solver, ∇::Function, u0::Vector{Float64})
    function loss_function(parameters)  
        local solution = run_solver(solver(), ∇, u0, parameters); #! Naming
        local ΔC14 = (solution .- u0[2]) ./ u0[2] .* 1000;  # Calculating ΔC14
        local miyake = DataFrame(CSV.File("Miyake12.csv")); # Reading Miyake data
        local ΔC14 .+= mean(miyake.d14c[1:4]);  # Calculates the ticktack offset
        local χ² = sum(((miyake.d14c .- ΔC14[1:28]) ./ miyake.sig_d14c) .^ 2);  # Calculating χ² 
        return -0.5 * χ²
    end
    return loss_function
end

select_gradient_solver

In [59]:
TO, P = read_hd5("Guttler14.hd5");      # Reading the data into the scope 

params = Vector{Float64}(undef, 6); # Storing the model params 
params[1] = 7.044873503263437;      # The mean position of the sinusoid 
params[2] = 0.18;                   # The modulation of the sinusoid w. r. t the mean
params[3] = 11.0;                   # Setting period of the sinusoid 
params[4] = 1.25;                   # The phase shift of the sinusoid
params[5] = 120.05769867244142;     # The height of the super gaussian 
params[6] = 12.0;                   # Width of the super-gaussian 

# The parameters to the model must be incorrect
production(t, params) = params[1] * (1 + params[2] * 
    sin(2 * π / params[3] * t + params[4])) +           # Sinusoidal production 
    params[5] * exp(- (params[6] * (t - 775)) ^ 16);    # Super Gaussian event
derivative(x, params, t) = vec(TO * x + production(t, params) * P);  # The derivative of the system 

u0 = TO \ (- params[1] * P);  # Equilibriating the system

burnproblem = ODEProblem(derivative, u0, (-360.0, 760.0), params);  # Burn in problem  
burnsolution = solve(burnproblem, reltol=1e-6).u[end];              # Running model 

5.859478950500488


In [80]:
function profile_forward_gradients(solvers, ∇::Function, u0::Vector{Float64}, params::Vector)
    local results = DataFrame(solvers = @.string(solvers));     # Net results
    local time_mean = Vector{Float64}(undef, length(solvers));  # Storage for speeds
    local time_var = Vector{Float64}(undef, length(solvers));   # Storing speed varience
    local gradient_matrix = Matrix{Float64}(undef, length(solvers), length(params)); # Storing gradients
    for (index, solver) in enumerate(solvers)
        local time_sample = Vector{Float64}(undef, 10);   # Storing the time sample
        for i in 1:10
            local timer = time();   # Creating a time
            local loss = select_gradient_solver(solver, ∇, u0); # Preparing the function
            local gradient = ForwardDiff.gradient(loss, params);    # Calculating the parameter gradient
            time_sample[i] = time() - timer;    # Appending the time to the time vector
            if i == 10  # Storing the gradient of the final run
                gradient_matrix[index, 1:end] = gradient;   # Appending the final gradient 
            end
        end
        time_mean[index] = mean(time_sample[2:end]);   # Calculating and storing the mean
        time_var[index] = var(time_sample[2:end]);     # Calculating and storing the varience
    end
    gradient_matrix = (gradient_matrix' .- median(gradient_matrix, dims=1)')';   # Deviations 
    gradient_matrix = @.abs(gradient_matrix); # Absolute deviations

    results.time_means = time_mean; # Storing the time means 
    results.time_vars = time_var;   # Storing the errors
    results.accuracy = vec(mean(gradient_matrix, dims=2));      # Accuracy as a mean of the deviations 
    results.accuracy_var = vec(var(gradient_matrix, dims=2));   # Accuracy error
    
    return results
end

# It would be dope to construct this framework only once and then be able to hand it only the two lines that are different. I think there must be a way to do this using macros

profile_forward_gradients (generic function with 2 methods)

Profiling the solvers by running each solver over the 30 year period surrounding the Miyake event. Saving the data to a file upon completion. 

I need to look into `BS5` as well as combinations like `AutoTsit5(Rosenbrook23)`. It may also be more efficient to have a function called write to CSV or something like that since I will be reusing the code in both places. I might just amalglomate the gradient and solver profiling into a single function although it would be much more elegant to have it as a macro or as a function programming element, where I passed a function kernel and retrieved the correct profiling function as the return output.

To consolidate my plan. I am going to write a function that returns the correct function code as an output for the profiler allowing me to severely reduce the amount of code that I have. This will be good at the loss of readibility and generality. I don't need either of those things in large amounts so we are good. I will also move the plotting function into this notebook and make it a function so that. I can visualise my results here. Finally, I will make a write CSV file function that will have mutliple modes, for when I want to append additional solvers.

In [81]:
solvers = [TRBDF2, BS3, Tsit5, Rosenbrock23,
    ROS34PW1a, QNDF1, ABDF2, ExplicitRK, DP5,
    TanYam7, Vern6];
profiles = profile_forward_gradients(solvers, derivative, burnsolution, params); # Running the solvers 

"gradient_profiles.csv"

In [ ]:
"""
Writes the results to a CSV file. If mode is set to append then the results are added to an existing CSV file without checking. Else the file is created with `append=false`.
"""
function write_csv(mode::String, data::DataFrame)::Nothing
    if mode == "append";                           # Checking for the .csv file 
        CSV.write("gradient_profiles.csv", profiles, append=true);# Adding new solvers to the CSV
    else 
        CSV.write("gradient_profiles.csv", profiles, append=false);# Creating the file if it does not exist 
    end
end